In [1]:
from dotenv import load_dotenv
from mysql.connector import Error
from DB_utils import *
import pandas as pd
import logging
import os
import time 

load_dotenv()  

True

In [2]:
logging.basicConfig(
    filename='logs/db_operations.log',       # Log file name
    level=logging.INFO,                      # Log level (DEBUG, INFO, WARNING, ERROR, CRITICAL)
    format='%(asctime)s - %(levelname)s - %(message)s',
    filemode='a'                        
)

In [ ]:
def load_csv_to_mysql(conn, csv_file, table_name):
    try:
        cursor = conn.cursor()
        query = f"""
        LOAD DATA LOCAL INFILE '{csv_file}'
        INTO TABLE `{table_name}`
        FIELDS TERMINATED BY ','
        ENCLOSED BY '"'
        LINES TERMINATED BY '\\n'
        IGNORE 1 ROWS;
        """
        cursor.execute(query)
        conn.commit()
        cursor.close()
        print(f"Loaded {csv_file} into {table_name}")
    except Error as e:
        print(f"Error loading CSV {csv_file}:", e)


In [7]:
def load_raw_data(conn):
    logging.info("-------Starting Ingestion-------")
    for file in os.listdir("csv_data"):
        start = time.time()
        if file.endswith(".csv"):
            file_path = 'csv_data/' + file
            df = pd.read_csv(file_path)
            logging.info(f"Uploading {file} to database...")
            create_table_from_csv(conn, file_path, file[:-4])
            load_csv_to_mysql(conn, file_path, file[:-4])
        end = time.time() 
        logging.info(f"Time taken to ingest {file}: {(end - start)/60:.2f} mins")  
    logging.info("-------Ingestion Complete-------")  
    

In [8]:
conn = ingest_db()
query =  f"CREATE DATABASE IF NOT EXISTS {os.getenv("DB_NAME")};"
execute_query(conn, query)
conn = ingest_db(db_name=os.getenv("DB_NAME"))


In [75]:
load_raw_data(conn)

Table `begin_inventory` created successfully.
Loaded csv_data/begin_inventory.csv into begin_inventory
Table `end_inventory` created successfully.
Loaded csv_data/end_inventory.csv into end_inventory
Table `purchases` created successfully.
Loaded csv_data/purchases.csv into purchases
Table `purchase_prices` created successfully.
Loaded csv_data/purchase_prices.csv into purchase_prices
Table `sales` created successfully.
Loaded csv_data/sales.csv into sales
Table `vendor_invoice` created successfully.
Loaded csv_data/vendor_invoice.csv into vendor_invoice
